In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns

In [2]:
data = pd.read_csv('radar_parameters.csv')
data.head()

,Unnamed: 0,Zh (dBZ),Zdr (dB),Ldr (dB),Kdp (deg km-1),Ah (dBZ/km),Adr (dB/km),R (mm/hr)
0,0,23.144878,0.418637,-41.757733,0.005395,0.000290,0.000012,2.393520
1,1,22.737156,0.322850,-43.772069,0.005194,0.000360,0.000012,3.502699
2,2,26.869826,0.330948,-43.577399,0.013385,0.000903,0.000030,8.627561
3,3,28.540561,0.399480,-42.139731,0.018872,0.001036,0.000043,8.424447
4,4,30.500127,0.543758,-39.763087,0.027438,0.001157,0.000064,8.189291


In [3]:
Z = (10**data['Zh (dBZ)'])**(1/10) #solve for Z from dBz using dBZ = 10log(Z) 
R_base = (Z/200)**(1/1.6) #solve for baseline rain rate using Z = 200R**1.6

In [4]:
#add those columns to dataframe to ensure they line up with the proper index when we split the data later
data['R_base'] = R_base
data['Z'] = Z

In [5]:
data.head(2)

,Unnamed: 0,Zh (dBZ),Zdr (dB),Ldr (dB),Kdp (deg km-1),Ah (dBZ/km),Adr (dB/km),R (mm/hr),R_base,Z
0,0,23.144878,0.418637,-41.757733,0.005395,0.00029,0.000012,2.393520,1.019556,206.294563
1,1,22.737156,0.322850,-43.772069,0.005194,0.00036,0.000012,3.502699,0.961454,187.808651


In [6]:
# Create feature matrix and target array
features = data.drop('R (mm/hr)', axis=1)

target = data['R (mm/hr)']

1) Split the data into 70-30 chunks for training and testing

In [7]:
Ftrain, Ftest, Ttrain, Ttest = train_test_split(features, target,
                                                random_state=1)

In [8]:
Ftrain.head(1)

,Unnamed: 0,Zh (dBZ),Zdr (dB),Ldr (dB),Kdp (deg km-1),Ah (dBZ/km),Adr (dB/km),R_base,Z
4116,4116,41.956799,1.550639,-31.883942,0.22322,0.003664,0.000647,15.281156,15692.059091


In [9]:
#Separate R baseline and calculated Z values from the feature matrices for training and testing sets
#so they are not taken into account for the linear regression
R_base_train = Ftrain['R_base']
Ftrain_new = Ftrain.drop(['R_base', 'Z'], axis=1)
R_base_test = Ftest['R_base']
Ftest_new = Ftest.drop(['R_base', 'Z'], axis=1)

In [10]:
Ftrain_new.head(1)

,Unnamed: 0,Zh (dBZ),Zdr (dB),Ldr (dB),Kdp (deg km-1),Ah (dBZ/km),Adr (dB/km)
4116,4116,41.956799,1.550639,-31.883942,0.22322,0.003664,0.000647


2) Train/validate a linear regression model using data from 1 and compare R2 and RMSE values for the model vs. test target values and baseline rain rate calculation vs. test target values to see which predicts the target values better.

In [11]:
from sklearn.linear_model import LinearRegression #choose model class

model = LinearRegression(fit_intercept=True) #instantiate model

model.fit(Ftrain_new, Ttrain) #fit model to data

lin_pred = model.predict(Ftest_new) #test using new data

In [12]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

lin_r2 = r2_score(lin_pred, Ttest) #calculate R2 between target test data and model
base_r2 = r2_score(R_base_test, Ttest) #calculate R2 between baseline prediction equation and model
lin_RMSE = mean_squared_error(lin_pred, Ttest,  squared=False)
base_RMSE = mean_squared_error(R_base_test, Ttest, squared=False)

print('Model R2 = ',lin_r2)
print('Baseline prediction calculation R2 = ', base_r2)
print('Model RMSE = ', lin_RMSE, '(mm/hr)') 
print('Baseline prediction calculation RMSE = ', base_RMSE, '(mm/hr)')

Model R2 =  0.9905267626588338
Baseline prediction calculation R2 =  0.2249639377726802
Model RMSE =  0.8817146505998322 (mm/hr)
Baseline prediction calculation RMSE =  7.06713305880775 (mm/hr)


Based on the score values, the model did a much better job predicting the target values than a baseline rain rate calculation

3) Create best polynomial model using grid search over orders 0-21 and cross-validation of 7 folds

In [13]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),
                         LinearRegression(**kwargs))

In [26]:
from sklearn.model_selection import GridSearchCV

param_grid = {'polynomialfeatures__degree': np.arange(5)} #ran overnight and did not finish all 21 orders so reducing to the most I can compute

grid = GridSearchCV(PolynomialRegression(), param_grid, cv=7)

In [27]:
grid.fit(Ftrain_new, Ttrain)

GridSearchCV(cv=7,
             estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('linearregression',
                                        LinearRegression())]),
             param_grid={'polynomialfeatures__degree': array([0, 1, 2, 3, 4])})

In [28]:
grid.best_params_

{'polynomialfeatures__degree': 2}

In [30]:
poly_model = grid.best_estimator_

poly_pred = poly_model.predict(Ftest_new) #test using new data

In [31]:
poly_r2 = r2_score(poly_pred, Ttest) #calculate R2 between target test data and model
poly_RMSE = mean_squared_error(poly_pred, Ttest,  squared=False)

print('Model R2 = ',poly_r2)
print('Model RMSE = ', poly_RMSE, '(mm/hr)') 

Model R2 =  0.9994057385976582
Model RMSE =  0.21856732463459472 (mm/hr)


4. Repeat with a Random Forest Regressor, performing a grid search utilizing provided parameters

In [46]:
from sklearn.ensemble import RandomForestRegressor
# Ran this one overnight and on Google Colab as well and wasn't able to complete, so reducing the gridsearch parameters to try to get something to complete
forest_parameters = {'bootstrap': [True, False],  
               'max_depth': [10, 20, 30,  None], # 40, 50, 60, 70, 80, 90, 100,
               'max_features': ['auto', 'sqrt'],  
               'min_samples_leaf': [1, 2, 4],  
               'min_samples_split': [2, 5, 10],  
               'n_estimators': [200, 400]} # 600, 800, 1000, 1200, 1400, 1600, 1800, 2000

forest_grid = GridSearchCV(RandomForestRegressor(), forest_parameters)

In [47]:
forest_grid.fit(Ftrain_new, Ttrain)

/Users/shelbyfatcheric/anaconda3/envs/ATMS_523/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
720 fits failed out of a total of 1440.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
720 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/shelbyfatcheric/anaconda3/envs/ATMS_523/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/shelbyfatcheric/anaconda3/envs/ATMS_523/lib/python3.12/site-packages/sklearn/base.py", line 1144, in wrapper
    estimator._validate_params()
  File "/Users/shelbyfatcheric/anaconda3/envs/ATMS_523/lib/python3.12/site-packages

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'bootstrap': [True, False],
                         'max_depth': [10, 20, 30, None],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [200, 400]})

In [48]:
forest_model = forest_grid.best_estimator_

forest_pred = forest_model.predict(Ftest_new) #test using new data

In [49]:
forest_r2 = r2_score(forest_pred, Ttest) #calculate R2 between target test data and model
forest_RMSE = mean_squared_error(forest_pred, Ttest,  squared=False)

print('Model R2 = ',forest_r2)
print('Model RMSE = ', forest_RMSE, '(mm/hr)') 

Model R2 =  0.9838020333650129
Model RMSE =  1.1101228814583326 (mm/hr)
